# An introduction on sampling via Langevin Dynamics

Our goal in this notebook is to show how to sample from a distribution with unknown normalization factor via Langevin dynamics. This approach is very rich and draws a lot of connections with optimization. As such, we will see that it can be used for both unconstrained and constrained sampling, by using similar tricks as in optimization (e.g., using mirror descent).

# 1- Warm-up: Unadjusted Langevin Algorithm

To begin with, you will first implement the most standard form of Langevin-based sampling methods, which is the Unadjusted Langevin Algorithm (ULA). This algorithm is designed by discretizing the SDE . The update rule can be written in the following form:

$x^{k+1} = x^k - \gamma_k \nabla V(x^k) + \sqrt{2\beta_k} \xi^k$,

where $\xi \sim \mathcal{N}(0,I)$ is a sample from a standard centered Gaussian distribution.

It turns out that ULA also works when the log distribution is not smooth, by replacing the gradient $\nabla V$ by a subgradient $\partial V$:

$x^{k+1} = x^k - \gamma_k \partial V(x^k) + \sqrt{2\beta_k} \xi^k$

In this first exercise, you will be asked to implement ULA for sampling from the distribution $d\mu(x) = \frac{1}{2}e^{-|x|}dx$.

Question 1.1: Show that the distribution $\mu$ is log-concave.

In [ ]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

In [ ]:
V = lambda x: np.abs(x)
grad_V = lambda x: np.sign(x)
pdf = lambda x: 0.5 * np.exp(-V(x))

In [ ]:
def ULA(T, x0 = 5, beta_0 = 1):
    x_tab = np.zeros(T)
    x = x0
    for t in range(T):
        # Your code here (use step size beta^k = beta_0 / sqrt{t+1})
        
        
        x_tab[t] = x
    return x_tab

In [ ]:
x_0 = 8 # Initial point
beta_0 = 1
N = 10000 # number of samples
T = 200 # Number of time steps of ULA

x_tab = np.zeros([N,T])

for n in range(N):
    x_tab[n,:] = ULA(T)

The following function computes the discrete TV distance between a histogram and a probability density function.

In [ ]:
def TV(hist, pdf):
    return np.sum(np.abs(hist[0] - pdf(0.5*(hist[1][1:] + hist[1][:-1])))) * (hist[1][1] - hist[1][0])

In [ ]:
x_ = np.linspace(-10,10, 101)
TV_tab = []
t_tab = []
for t in range(10, T, int(T/10)):
    plt.figure()
    hist = plt.hist(x_tab[:,t], bins=200, normed = True)
    plt.plot(x_, 0.5 * np.exp(-V(x_)))
    plt.show()
    t_tab.append(t)
    TV_tab.append(TV(hist, pdf))

In [ ]:
plt.loglog(t_tab, TV_tab,  '+')
plt.ylabel("TV distance")
plt.xlabel("Iterations")
plt.show()

The stochastic variant of ULA, called Stochastic Gradient Langevin Dynamics, assumes that only unbiased estimates of the gradient are available, instead of the true gradient.

Question 1.2: Try to add random noise to the gradient at each iteration and observe the effect on the convergence.

# 2- Mirrored Langevin Dynamics

Mirrored Langevin Dynamics (MLD) is a sampling algorithm for sampling from distributions that are constrained to convex compact sets. For more details and a more complete description of MLD, please refer to our paper at https://arxiv.org/pdf/1802.10174.pdf .

Similarly as in mirror descent for optimization, the goal of MLD is to map the primal variables to a dual space, in which the variables are unconstrained. ULA is then used in the dual space in order to sample from the dual unconstrained distribution. Finally, mapping the samples back using the inverse mirror map, it allows for sampling in the primal space.

More precisely, given a distribution $\mu$ and a mirror map $h$ satisfying certain assumptions, we can construct the push-forward of $\mu$ under $\nabla h$, denoted $\nu = \nabla h \# \mu$ and defined as:
\begin{equation*}
\nu = \nabla h \# \mu  \Leftrightarrow \nu (A) = \mu(   \nabla h^{-1}(A) ), \ A \text{   Borel}.
\end{equation*}

Therefore, if we are able to sample from the unconstrained distribution $\nu$, then, using the inverse mirror map $\nabla h^{-1}$, we can obtain samples from $\mu$. 



In this notebook, we will consider the particular case of sampling a distribution $d\mu(x) \propto e^{-V(x)}dx$, $x \in \mathbb{R}^d$ constrained over the Simplex $\Delta_{d} = \{x \in \mathbb{R}^{d} | \sum_{i=1}^{d} x_i \leq 1, x_i \geq 0\}$.

In the case of the Simplex, a good mirror map is given by the entropic mirror map:

\begin{equation}
h(x) = \sum_{i=1}^d x_i \log x_i + \left(1- \sum_{i=1}^d x_i \right)\log\left(1- \sum_{i=1}^d x_i \right).
\end{equation}

It is then possible to show that $d\nu = \nabla h \# e^{-V(x)}dx = e^{-W(y)}dy$ where the potential $W$ is given by:

\begin{equation}
W(y) = V \circ \nabla h^{\star}(y) - \sum_{i=1}^d y_i + (d+1) h^\star(y),
\end{equation}

where $h^\star(y) = \log \left(1 + \sum_{i=1}^d e^{y_i}\right)$ is the Fenchel dual of $h$.

Using this expression for the unconstrained distribution $\nu$, it is thus possible to apply ULA to obtain samples from $\nu$. Then, mapping back to the primal space, we can obtain samples from $\mu$.


The update rule for MLD is thus the following:
\begin{align*}
y^{k+1} &= y^k - \beta_k \nabla W(y^k) + \sqrt{2\beta_k} \xi^k \\
x^{k+1} &= \nabla h^\star(y^{k+1})
\end{align*}

where $\nabla h^\star$ is the gradient of $h^\star$ and it can be shown that $\nabla h^\star = (\nabla h)^{-1}$.

You will now be asked to implement MLD in order to sample from a Dirichlet distribution. The probability density function of the Dirichlet distribution with parameter $\{\alpha_1, \ldots, \alpha_{d+1}\}$ is
\begin{equation}
p(x) = \frac{1}{C} \Pi_{i=1}^{d+1} x_i^{\alpha_i - 1}
\end{equation}
where $C$ is the unknown normalization factor, and $x_{d+1} = 1 - \sum_{i=1}^d x_i$.

The negative log density is thus given by:
\begin{equation}
V(x) = -\log(p(x)) = \log(C) - \sum_{i=1}^{d+1} (\alpha_i + 1) \log(x_i)
\end{equation}

Together with the entropic mirror map, the dual potential becomes:
\begin{equation}
W(y) = \log(C) - \sum_{i=1}^{d} \alpha_i y_i + \left(\sum_{i=1}^{d+1} \alpha_i \right) h^\star(y)
\end{equation}

Question 2.1: Show that W is convex, implying that the dual distribution is log-concave.

Question 2.2: Show that $\nabla h^\star = \frac{e^y}{1+\sum_{i=1}e^{y_i}}$ and $\nabla W(y) = \alpha' - \left(\sum_{i=1}^{d+1} \alpha_i \right) \frac{e^y}{1+\sum_{i=1}e^{y_i}}$, where $\alpha' = \{\alpha_1, \ldots, \alpha_d\}$.

Task: Implement MLD for sampling from a $11$-dimensional Dirichlet distribution with parameters $\alpha = (100,50,10,1,1,1,1,1,1,1,1)$. The sampling should only be performed over the $d=10$ first variables in $\Delta_d$ since $x_{11} = 1- \sum_{i=1}^{10} x_i$.

For computational reasons, we will simply compute the TV distance for the marginalized distribution of the Dirichlet distribution with respect to the first variable.

In [ ]:
y0 = 0
beta_0 = 0.1
grad_h_star = lambda y: # Your code here

In [ ]:
d = 10
alpha = np.array([100,50,10,1,1,1,1,1,1,1,1])
pdf_beta = lambda x: scipy.stats.beta.pdf(x, alpha[0], np.sum(alpha[1:])) # true marginal density distribution of x_1

In [ ]:
def MLD(T, y0 = 0, beta_0 = 0.1):
    # save_dim[t] should only contain the first coordinate of the sample x at iteration t
    save_dim = np.zeros(T)
    y = y0
    for t in range(T):
        # Your code here ((use step size beta^k = beta_0 / sqrt{t+1}))
        # y = 
        
        x = grad_h_star(y)
        save_dim[t] = x[0] # only save first coordinate of the sample vector
    return save_dim

In [ ]:
N = 1000
T = 100
save_dim = np.zeros([N,T])

for n in range(N):
    save_dim[n,:] = MLD(T)

In [ ]:
x_axis = np.linspace(0,1,100)
TV_tab = []
t_tab = []
for t in range(10, T, int(T/20)):
    plt.figure()
    hist = plt.hist(save_dim[:,t], bins=100, normed = True)
    plt.plot(x_axis, pdf_beta(x_axis))
    plt.show()
    t_tab.append(t)
    TV_tab.append(TV(hist, pdf_beta))

In [ ]:
plt.loglog(t_tab, TV_tab,  '+')
plt.ylabel("TV distance")
plt.xlabel("Iterations")
plt.show()

# 3- Application to Latent Dirichlet Allocation

In this section, we present the application of MLD to Latent Dirichlet Allocation (LDA). Because of the complexity of notations, no implementation is asked to you, but you may want to run and look at the implementation we provide in order to have an idea of how MLD is applicable to this case.

In the context of machine learning, LDA is a graphical, generative model that explains how the words in a group of documents are generated based on some ''latent''' topic model.

In the most basic sense, the LDA model assumes that there are latent features, which we define as topics. Topics are distributions over words, defined over a probability simplex. Then, the documents are generated as mixtures of such topics. As a results, every word in a documents is given a topic assignment, denoting that it is drawn from the the distribution of that particular topic. 

The graphical model below summarizes the overall structure of the approach:


<img src="LDA graphical model.png">


Assume that we have $D$ documents in the corpus, $W$ words in our "vocabulary" and $K$ latent topics. Please note that number of topics to be chosen is somewhat a hyperparameter that should vary across different datasets. In our application, we will consider a constant $K$. Then, $\pi \in \mathbb R^{K \times W}$ is the topic distributions over words, which is drawn from a Dirichlet prior with parameter $\beta$, $\eta \in \mathbb R^{D \times K}$ is the mixing proportions of topics per document, which is sampled from a Dirichlet prior with parameter $\alpha$, $z_{di}$ denotes the topics assignment in document $d$ for word $i$, and $w_{di}$ is the i$^{th}$ word in d$^{th}$ document.

In the light of these information let us introduce the joint distribution that the graphical model given above represents:

$p(w, z, \eta, \pi | \alpha, \beta) = \prod_{d=1}^{D} \frac{\Gamma(K \alpha)}{\Gamma(\alpha)^K} \Pi_{k=1}^{K} \eta_{dk}^{\alpha - 1 + n_{dk.}} \prod_{k=1}^{K} \frac{\Gamma(W \beta)}{\Gamma(\beta)^W} \Pi_{w=1}^{W} \pi_{kw}^{\beta -1  + n_{.kw}} $,

where $n_{dkw} = \# \{ i : w_{ij} = w, z_{ij} = k \}$. Dot represents summation over that index.


Next, we will dive into particular details of the joint distirbution and we will discuss how we could represent it by especially re-parametrizing the probability simplex that the topic distributions are defined on.



## Why do we sample?

Especially in the context of LDA, sampling methods make great sense. In the end we learn some joint distribution and somehow marginalize this joint distribution with respect to some parameter, which are the topic distributions in our application. We can sample ''models'' from the posterior distribution and a better sampling performance means a better model.

Obviously we cannot directly make use of the joint distribution in its current form and it needs to be simplified. Initial trick that we are going to use is that we will integrate the joint distribution with respect to $\eta$ to obtain:

$p(w, z, \pi | \alpha, \beta) = \prod_{d=1}^{D} \frac{\Gamma(K \alpha)}{\Gamma(K \alpha + n_{d..})} \prod_{k=1}^{K} \frac{\Gamma(\alpha + n_{dk.})}{\Gamma(\alpha)} \prod_{k=1}^{K} \frac{\Gamma(W \beta)}{\Gamma(\beta)^W} \prod_{w=1}^{W} \pi_{kw}^{\beta -1 + n_{.kw}} $


## Our Task

In this execise, our main goal is to use MLD sampling schemes to sample from the log posterior distribution of the parameters, namely topic distirbution over the words.

We will use the properties of conditional probability to compute the posterior distirbution of $\pi_k$s. For the graphical model above, we could infer the following two quantities:

- $p(w, z, \pi | \alpha, \beta) = \prod_{d=1}^{D} \frac{\Gamma(K \alpha)}{\Gamma(K \alpha + n_{d..})} \Pi_{k=1}^{K} \frac{\Gamma(\alpha + n_{dk.})}{\Gamma(\alpha)} \prod_{k=1}^{K} \frac{\Gamma(W \beta)}{\Gamma(\beta)^W} \Pi_{w=1}^{W} \pi_{kw}^{\beta - 1 + n_{.kw}} $

- $p(w, z | \alpha, \beta) = \prod_{d=1}^{D} \frac{\Gamma(K \alpha)}{\Gamma(K \alpha + n_{d..})} \Pi_{k=1}^{K} \frac{\Gamma(\alpha + n_{dk.})}{\Gamma(\alpha)} \prod_{k=1}^{K} \frac{\Gamma(W \beta)}{\Gamma(W \beta + n_{.k.})} + \Pi_{w=1}^{W} \frac{\Gamma(\beta + n_{.kw})}{\Gamma(\beta)} $

Then,  we can compute $p(\pi | \alpha, \beta)$.

## Understanding the Implementation Details

In the implementation, there are a few points that requires explanation:

- Bkw_mean: This is one of the key variables that is required to compute gradients of the log posterior. It is a $K \times W$ dimensional matrix that stores the ''Gibbs-sampled'' number of occurences of each word $w$ generated by topic $k$. Observe the relationship between $n_{dkw}$ and Bkw_mean variable.

- Normalization factor: Since we train using mini batches, keep in mind to ''scale'' the quantitied with number of all documents in Wikipedia / mini-batch size.

- Step size: In LD sampling schemes, step size appears as a scaling factor that controls the variance of the Gaussian noise



In [ ]:
from __future__ import division

import errno
import numpy as np
import os
import time
import util_funcs
# from bigfloat import *

# Class definition for AMLD sampling scheme
class AMLDSampler(object):
    
    def __init__(self, D, K, W, alpha, beta, theta0,
            step_size_params, init_batch, results_dir, n_articles):
        self.D = D # num of documents
        self.K = K # num of topics
        self.W = W # num of words in vocabulary
        self.alpha = alpha
        self.beta = beta
        self.step_size_params = step_size_params
        self.update_ct = 0
        self.n_articles = n_articles
        self.results_dir = results_dir
        self.theta = theta0
        self.phi = theta0 / np.sum(theta0,1)[:,np.newaxis]

        # get Gibbs sampled counts for estimating initialization
        (names, train_cts, test_cts) = zip(*init_batch)
        train_cts = list(train_cts)
        batch_D = len(train_cts)
        (A_dk, B_kw) = self.sample_counts(train_cts, batch_D, 100)

        # initilaize close to optimal in the dual
        y = np.zeros( (K, W-1) )
        for k in range(K):
            # y[k, :] = np.log( self.phi[k, 0:-1] ) - np.log(1. -  np.sum( self.phi[k, 0:-1] ) )
            y[k, :] = np.log( ( self.beta + B_kw[k, 0:-1] ) / ( B_kw[k, -1] + self.beta ) )
        self.y = y

        # retrieve primal variable from dual (recall the entropic mirror map)
        phi = np.zeros( (K, W) )
        for k in range(K):
            phi[k, 0:-1] = np.exp( y[k, :] ) / ( 1. + np.sum( np.exp( y[k, :] ) ) )            
            phi[k, -1] = 1. - np.sum( phi[k, 0:-1] )
        self.phi = phi

    # Gibbs sampling. We use this to sample topic assignments
    def sample_counts(self, train_cts, batch_D, num_samples):
        batch_N = sum(sum(ddict.values()) for ddict in train_cts)
        uni_rvs = np.random.uniform(size = (batch_N)*(num_samples+1))
        z  = [{} for d in range(0, batch_D)]
        Adk = np.zeros((batch_D, self.K), dtype = np.uint32)
        Bkw = np.zeros((self.K, self.W), dtype = np.uint32)
        Adk_mean = np.zeros(Adk.shape)
        Bkw_mean = np.zeros(Bkw.shape)
        burn_in = num_samples // 2
        util_funcs.sample_z_ids(Adk_mean, Bkw_mean, Adk, Bkw, self.phi,
                uni_rvs, train_cts, z, self.alpha, num_samples, burn_in)
        return (Adk_mean, Bkw_mean)

    def update(self, train_cts, num_samples):
        batch_D = len(train_cts)
        Adk_mean, Bkw_mean = self.sample_counts(train_cts, batch_D, num_samples)

        phi_star = self.phi.copy()
        y_star = self.y.copy()
        (a,b,c) = self.step_size_params
        eps_t  = a*((1 + self.update_ct/b)**-c)

        for k in range(self.K):
            phi_k = self.phi[k,:]
            y_k = self.y[k,:]
            z = np.random.randn(self.W - 1)
            norm_factor = (3068000. / sum(sum(cntr.values()) for cntr in train_cts))

            grad = ( -self.beta - norm_factor *  Bkw_mean[k,0:-1] ) + ( ( self.W * self.beta + norm_factor * np.sum(Bkw_mean[k,:]) ) * phi_k[0:-1] )
            y_k = y_k - eps_t * grad + (2*eps_t)**.5*z

            y_star[k, :] = y_k    
            phi_star[k, 0:-1] = np.exp( y_k ) / ( 1. + np.sum( np.exp( y_k ) ) )
            phi_star[k, -1] = 1. - np.sum( phi_star[k, 0:-1] )
        self.phi = phi_star
        self.y = y_star
        self.update_ct += 1
        return (Adk_mean, Bkw_mean)

    def run_online(self, num_updates, samples_per_update, batched_cts,
            holdout_train_cts, holdout_test_cts):
        self.log_preds = []
        t = 1
        self.ho_log_preds = []
        self.avg_probs = {(d, w): 0.0 for (d, ctr) in enumerate(holdout_test_cts) for w in ctr}
        self.ho_count = 0
        self.create_output_dir()
        (a,b,c) = self.step_size_params
        params_dict = {'a':a,'b':b,'c':c,'alpha':self.alpha,'beta':self.beta,
                'K':self.K,'samples':samples_per_update,'func':'online'}
        sampler_name = self.__class__.__name__
        for batch in batched_cts:
            if t == num_updates:
                break
            params_dict['batch_size'] = len(batch)
            self.basename = str(params_dict)
            (names, train_cts, test_cts) = zip(*batch)
            train_cts = list(train_cts)
            test_cts = list(test_cts)
            start = time.time()
            (Adk_mean, Bkw_mean) = self.update(train_cts, samples_per_update)
            self.log_preds.append(self.log_pred(test_cts, Adk_mean,Bkw_mean))
            end = time.time()
            docs_so_far = t * len(batch)
            # print sampler_name + " iteration %d, docs so far %d, log pred %g, time %g" % (t, docs_so_far, self.log_preds[t-1], end - start)
            # Assess holdout log-pred every 5000 documents
            if docs_so_far % 2000  == 0:
                # print 'holdout probability after %d documents' % (docs_so_far)
                ho_lp = self.holdout_log_pred(holdout_train_cts, holdout_test_cts, samples_per_update)
                self.ho_log_preds.append([docs_so_far, ho_lp])
                self.save_variables(['ho_log_preds'])
                print('Holdout Probability ( %d documents ): %g' % (docs_so_far, ho_lp))
            t += 1
        return self.ho_log_preds

    def log_pred(self, test_cts, Adk_mean, Bkw_mean):
        eta_hat = Adk_mean + self.alpha
        eta_hat /= np.sum(eta_hat, 1)[:, np.newaxis]
        phi_hat = self.phi
        log_probs = {(d, w): cntr[w]*np.log(np.dot(eta_hat[d, :],
                     phi_hat[:, w])) for (d, cntr) in enumerate(test_cts)
                     for w in cntr}
        num_words = sum(sum(cntr.values()) for cntr in test_cts)
        return sum(log_probs.values()) / num_words

    def holdout_log_pred(self, holdout_train_cts, holdout_test_cts, num_samps):
        batch_D = len(holdout_train_cts)
        Adk_mean, Bkw_mean = self.sample_counts(holdout_train_cts, batch_D,
                                                num_samps)
        eta_hat = Adk_mean + self.alpha
        eta_hat /= np.sum(eta_hat, 1)[:, np.newaxis]
        phi_hat = self.phi
        T = self.ho_count
        old_avg = self.avg_probs
        avg_probs = {(d, w): (T*old_avg[(d, w)] +
                              np.dot(eta_hat[d, :], phi_hat[:, w])) / (T+1)
                     for (d, w) in old_avg}
        self.avg_probs = avg_probs
        self.ho_count += 1
        log_avg_probs = {(d, w): cntr[w] * np.log(avg_probs[(d, w)])
                         for (d, cntr) in enumerate(holdout_test_cts) for w in cntr}
        num_words = sum(sum(cntr.values()) for cntr in holdout_test_cts)
        return sum(log_avg_probs.values()) / num_words

    def create_output_dir(self):
        self.dirname = os.path.join(self.results_dir, self.__class__.__name__)
        try:
            os.makedirs(self.dirname)
        except OSError as exc:
            if exc.errno == errno.EEXIST:
                pass
            else:
                raise

    def save_variables(self, attrs):
        for attr in attrs:
            np.savetxt(os.path.join(self.dirname,
                                    self.basename+attr+'_LD_' + str(self.n_articles) + '.dat'),
                       np.array(getattr(self, attr)))

    def store_phi(self, iter_num):
        try:
            self.stored_phis[str(iter_num)] = self.phi
        except AttributeError:
            self.stored_phis = {str(iter_num): self.phi}

    def save_stored_phis(self):
        np.savez(os.path.join(self.dirname, self.basename+'_stored_phis.npz'),
                 **self.stored_phis)



In [ ]:
from __future__ import division

import argparse
import itertools
import numpy as np
import os
import pickle

import processwiki as pw

home = os.path.expanduser('~')
np.set_printoptions(precision=3, suppress=True)

# Input parameters. Default values are provided in case they are not set by the user
alpha = 0.01
beta = 0.0001
K = 50
batch_size = 50
epsilon = 0.01
tau0 = 1000
kappa = 0.55
samples_per_update = 50
num_updates = 1000
output_dir = '.'

D = 3.3e6 # Approx number of documents in wikipedia

n_articles = int(2e4) # Number of articles we will have in our dataset
n_test_articles = int(1e3) # Number of test documents

vocab = pw.create_vocab('wiki.vocab') # Vocabulary is generated from an existing source
W = len(vocab) # Number of words in out vocabulary

# Read the dataset
with open('random_articles' + str(n_articles) + '.dms', 'rb') as infile:
    # random_articles = pickle.load(infile)
    u = pickle._Unpickler(infile)
    u.encoding = 'latin1'
    random_articles = u.load()


##### SMLD #####

#step size parameters
a = 1e-4
b = 1e4
c = 0.525

step_size_params = (a, b, c)
data = pw.parse_docs(random_articles, vocab)

# separate date into test and train
test_data = itertools.islice(data, n_test_articles)
(test_names, holdout_train_cts, holdout_test_cts) = zip(*list(test_data))
holdout_train_cts = list(holdout_train_cts)
holdout_train_cts = list(holdout_train_cts)

# automatically batch the data for training
batched_cts = pw.take_every(batch_size, data)
init_batch = next(batched_cts)

print("Running LD Sampler for AMLD, a = %g, b = %g, c = %g" % (a, b, c))
theta0 = np.random.gamma(1,1,(K,W))
ld_func = AMLDSampler(D, K, W, alpha, beta, theta0,
        step_size_params, init_batch, output_dir, n_articles).run_online
ld_args = (num_updates, samples_per_update, batched_cts,
        holdout_train_cts, holdout_test_cts)
log_pred_mld = ld_func(*ld_args)




The measure traditionally used for topic models is the \textit{perplexity} of held-out documents $w_d$ defined as:
\begin{equation}
perp(w_d) = \exp \left(\frac{\sum_{i=1}^{n_{d..}} \log p(w_{di})}{ n_{d..}} \right)
\end{equation}

In [ ]:
def compute_perplexity(value):
    return np.exp(-1 * value)

In [ ]:
import matplotlib.pyplot as plt

data_mld = np.zeros((len(log_pred_mld)))

iter_mld = np.zeros((len(log_pred_mld)))
    
for i in range(len(log_pred_mld)):
    temp = log_pred_mld[i]
    data_mld[i] = compute_perplexity(temp[1])
    iter_mld[i] = temp[0]
    
    
##### PLOT #####

plt.figure()
plt.plot(iter_mld, data_mld, 'r', linewidth=5)
plt.xlabel('iterations')
plt.ylabel('Perplexity')
plt.legend(['MLD'])
plt.show()

